
# Calculate Detector Counts: NEI
Compute the AIA response use our full emission model, including non-equilibrium ionization.

In [1]:
import os
import time
import h5py
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'
import numpy as np
import astropy.units as u
import sunpy.sun.constants
import matplotlib.pyplot as plt
import dask
import distributed

import synthesizAR
from synthesizAR.instruments import InstrumentSDOAIA
from synthesizAR.atomic import EmissionModel, Element
#from synthesizAR.interfaces import EbtelInterface

%matplotlib inline

In [2]:
cluster = distributed.LocalCluster(n_workers=64,threads_per_worker=1)
client = distributed.Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:46279 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 64 Cores: 64 Memory: 270.38 GB


Load in the desired field and emission model

In [3]:
field = synthesizAR.Field.restore('/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling/field_checkpoint/')

We are using an emission model which includes only the most dominant ions. Comparisons to the temperature response functions show these provide accurate coverage.

In [ ]:
em_model = EmissionModel.restore('/storage-home/w/wtb2/data/timelag_synthesis_v2/base_emission_model.json')

Compute and store the non-equilibrium ionization populations for each loop

In [ ]:
futures = em_model.calculate_ionization_fraction(field,
                '/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling/nei/ionization_fractions.h5',
                interface=EbtelInterface, client=client)

In [ ]:
em_model.save('/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling/nei/emission_model.json')

In [ ]:
futures = None

Or just reload the emission model

In [ ]:
em_model = EmissionModel.restore('/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling/nei/emission_model.json')

Compute the detector counts

In [5]:
aia = InstrumentSDOAIA([0,10000]*u.s, field.magnetogram.observer_coordinate)

/storage-home/w/wtb2/anaconda3/envs/synthesized-timelags/lib/python3.6/site-packages/sunpy-1.0.dev9869-py3.6-linux-x86_64.egg/sunpy/map/mapbase.py:645: Warning: Missing metadata for heliographic longitude: assuming longitude of 0 degrees
  lon=self.heliographic_longitude,


In [6]:
observer = synthesizAR.Observer(field,[aia],parallel=True)

In [7]:
observer.build_detector_files('/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling/nei/',
                              ds=0.5*u.Mm)

/storage-home/w/wtb2/anaconda3/envs/synthesized-timelags/lib/python3.6/site-packages/scipy/interpolate/_fitpack_impl.py:299: RuntimeWarning: The maximal number of iterations (20) allowed for finding smoothing
spline with fp=s has been reached. Probable cause: s too small.
(abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))
/storage-home/w/wtb2/anaconda3/envs/synthesized-timelags/lib/python3.6/site-packages/scipy/interpolate/_fitpack_impl.py:299: RuntimeWarning: A theoretically impossible result when finding a smoothing spline
with fp = s. Probable cause: s too small. (abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))


In [ ]:
futures_flat = observer.flatten_detector_counts(emission_model=em_model)

And finally build the maps

In [8]:
futures_bin = observer.bin_detector_counts(
    '/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling/nei/')